In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd "/content/gdrive/MyDrive/AI-Thinking-Data"

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1EH1SqwJ8QqFAhAXhp4Xs80aQOOfssre8/AI-Thinking-Data


In [ ]:
import pandas as pd
titanic = pd.read_csv('titanic_train.csv')

In [ ]:
titanic.sample()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
397,398,0,2,"McKane, Mr. Peter David",male,46.0,0,0,28403,26.0,NaN,S


In [ ]:
titanic["Age"]=titanic["Age"].fillna(titanic["Age"].mean())
titanic["Cabin"]=titanic["Cabin"].fillna('N')
titanic["Embarked"]=titanic["Embarked"].fillna('N')

In [ ]:
titanic['Age']

0      22.000000
1      38.000000
2      26.000000
3      35.000000
4      35.000000
         ...    
886    27.000000
887    19.000000
888    29.699118
889    26.000000
890    32.000000
Name: Age, Length: 891, dtype: float64

In [ ]:
for i in range(len(titanic)):
  titanic['Cabin'].iloc[i] = titanic["Cabin"].iloc[i][0]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
from sklearn.preprocessing import LabelEncoder

def fillna(df):
  df['Age'].fillna(df['Age'].mean(), inplace=True)
  df['Cabin'].fillna('N', inplace=True)
  df['Embarked'].fillna('N', inplace=True)
  df['Fare'].fillna(0, inplace=True)
  return df

def drop_features(df):
  df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
  return df

def format_feature(df):
  df['Cabin'] = df['Cabin'].str[:1]
  features = ['Cabin', 'Sex', 'Embarked']
  for feature in features:
    Ie = LabelEncoder()
    Ie = Ie.fit(df[feature])
    df[feature] = Ie.transform(df[feature])
  return df

def transform_features(df):
  df = fillna(df)
  df = drop_features(df)
  df = format_feature(df)
  return df


In [ ]:
titanic_df = pd.read_csv("./titanic_train.csv")
y_titanic_df=titanic_df["Survived"]
X_titanic_df = titanic_df.drop('Survived',axis=1) 


X_titanic_df = transform_features(X_titanic_df)

In [ ]:
X_titanic_df

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,22.000000,1,0,7.2500,7,3
1,1,0,38.000000,1,0,71.2833,2,0
2,3,0,26.000000,0,0,7.9250,7,3
3,1,0,35.000000,1,0,53.1000,2,3
4,3,1,35.000000,0,0,8.0500,7,3
...,...,...,...,...,...,...,...,...
886,2,1,27.000000,0,0,13.0000,7,3
887,1,0,19.000000,0,0,30.0000,1,3
888,3,0,29.699118,1,2,23.4500,7,3
889,1,1,26.000000,0,0,30.0000,2,0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, train_size=0.8, random_state=11)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
dtc=DecisionTreeClassifier()
dtc.fit(X_train,y_train)
dtc_pred=dtc.predict(X_test)
dtc_acc=accuracy_score(y_test, dtc_pred)

rtc=RandomForestClassifier()
rtc.fit(X_train,y_train)
rtc_pred=rtc.predict(X_test)
rtc_acc=accuracy_score(y_test, rtc_pred)

lr=LogisticRegression()
lr.fit(X_train,y_train)
lr_pred=lr.predict(X_test)
lr_acc=accuracy_score(y_test, lr_pred)

print(dtc_acc, rtc_acc, lr_acc)

0.8212290502793296 0.8491620111731844 0.8491620111731844


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
parameters = {'max_depth': [2,3,5,10],
              'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8]}

In [ ]:
from sklearn.model_selection import GridSearchCV
dtc = DecisionTreeClassifier(random_state=11)
grid_dtree = GridSearchCV(dtc, param_grid=parameters, cv=3, refit=True)

grid_dtree.fit(X_train, y_train)

scores_dtc = pd.DataFrame(grid_dtree.cv_results_)
scores_dtc[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 2, 'min_samples_leaf': 1, 'min_s...",0.773907,25,0.752101,0.801688,0.767932
1,"{'max_depth': 2, 'min_samples_leaf': 1, 'min_s...",0.773907,25,0.752101,0.801688,0.767932
2,"{'max_depth': 2, 'min_samples_leaf': 1, 'min_s...",0.773907,25,0.752101,0.801688,0.767932
3,"{'max_depth': 2, 'min_samples_leaf': 5, 'min_s...",0.773907,25,0.752101,0.801688,0.767932
4,"{'max_depth': 2, 'min_samples_leaf': 5, 'min_s...",0.773907,25,0.752101,0.801688,0.767932
5,"{'max_depth': 2, 'min_samples_leaf': 5, 'min_s...",0.773907,25,0.752101,0.801688,0.767932
6,"{'max_depth': 2, 'min_samples_leaf': 8, 'min_s...",0.773907,25,0.752101,0.801688,0.767932
7,"{'max_depth': 2, 'min_samples_leaf': 8, 'min_s...",0.773907,25,0.752101,0.801688,0.767932
8,"{'max_depth': 2, 'min_samples_leaf': 8, 'min_s...",0.773907,25,0.752101,0.801688,0.767932
9,"{'max_depth': 3, 'min_samples_leaf': 1, 'min_s...",0.793574,7,0.768908,0.856540,0.755274


In [ ]:
estimator=grid_dtree.best_estimator_

In [ ]:
print(grid_dtree.best_params_)

{'max_depth': 10, 'min_samples_leaf': 8, 'min_samples_split': 2}


In [ ]:
pred=estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

테스트 데이터 세트 정확도: 0.8659


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def get_clf_eval(y_test, pred):
  confusion = confusion_matrix(y_test, pred)
  accuracy = accuracy_score(y_test, pred)
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)
  print('오차 행렬')
  print(confusion)
  print('정확되 {0: .4f}, 정밀도: {1: .4f}, 재현율: {2: .4f}' .format(accuracy, precision, recall))

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

titanic_df = pd.read_csv('./titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=11)

lr_clf = LogisticRegression()

lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
get_clf_eval(y_test, pred)

오차 행렬
[[104  14]
 [ 13  48]]
정확되  0.8492, 정밀도:  0.7742, 재현율:  0.7869


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
  confusion = confusion_matrix(y_test, pred)
  accuracy = accuracy_score(y_test, pred)
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)
  f1 = f1_score(y_test, pred)

  roc_auc = roc_auc_score(y_test, pred_proba)
  print('오차 행렬')
  print(confusion)
  print('정확도: {0: .4f}, 정밀도: {1: .4f}, 재현율: {2: .4f}, ')